<a href="https://colab.research.google.com/github/MykolaPrykhodko/MykolaPrykhodko/blob/main/%D0%94%D0%B5%D0%BD%D0%B8%D1%81_Buget_FOP_2023.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Програма розрахунку бюджетних показників ФОП АТ "Украгазбанка"**

In [291]:
import pandas as pd
import numpy as np
from pathlib import Path

## Підключення хмарного середовища зберігання довідників для розрахунку

In [292]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


## Для читання потрібна певна авторізація

###Імпортування бібліотек для аутентифікації та встановлення зв'язку із Таблицями.

In [293]:
from google.colab import auth
auth.authenticate_user()

import gspread
from google.auth import default
creds, _ = default()

gc = gspread.authorize(creds)

### Відкриття файла із переліком персонала для обчислювання за посіланням (url)

In [294]:
path_in ='/content/gdrive/Shareddrives/Фінансовий департамент/Управління фінансового планування/Бюджет 2023/Budget FOP/In/staff.xlsx'

### Отримані дані перетворюємо на датафрейм, як дінамічну структуру даних для подальшого доповнення результатами розрахунків (як приклад - в консоль роздруковано перших 20 значень)

In [295]:
df_staff = pd.read_excel(path_in).astype({"tabNum": str}, errors='raise')
df_staff_for_merge = pd.read_excel(path_in).astype({"tabNum": str}, errors='raise')

In [296]:
print(df_staff.tail(10))

           tabNum      Name    CFOcode                CFOname
3722  вакансія_61   нач.від     1.57.   Департамент комплаєнс
3723  вакансія_62   нач.від     1.57.   Департамент комплаєнс
3724  вакансія_63  гол.спец     1.57.   Департамент комплаєнс
3725  вакансія_64   нач.від     1.57.   Департамент комплаєнс
3726       труд_1        НР  9.00.3.1.                     НР
3727       труд_2        НР  9.00.3.1.                     НР
3728       труд_3        НР  9.00.3.1.                     НР
3729       труд_4        НР  9.00.3.1.                     НР
3730       труд_5        НР  9.00.3.1.                     НР
3731       труд_6        НР  9.00.3.1.                     НР


## Допоміжні функції розрахунків

In [297]:
def avg_salary(path_in):
  p_3 = pd.read_excel(path_in)

  def calculate(num_tab):
    x = 1
    y = 7
    lst = []
    while y <= len(p_3.columns):
        avg = sum(p_3.iloc[num_tab, x:y])/6
        lst.append(avg)
        x += 1
        y += 1
    return lst
  
  global_lst = []
  for i in range(len(p_3)):
    global_lst.append(calculate(i))
    
  df = pd.DataFrame(global_lst)

  finish = pd.concat([p_3['tabNum'], df], axis=1).iloc[:,0:13]
  finish.rename(columns={0: 'січень'
  , 1: 'лютий'
  , 2: 'березень'
  , 3: 'квітень'
  , 4: 'травень'
  , 5: 'червень'
  , 6: 'липень'
  , 7: 'серпень'
  , 8: 'вересень'
  , 9: 'жовтень'
  , 10: 'листопад'
  , 11: 'грудень'}, inplace=True)
  finish=pd.melt(finish, id_vars='tabNum', value_vars=['січень',
                                                        'лютий',
                                                        'березень',
                                                        'квітень',
                                                        'травень',
                                                        'червень',
                                                        'липень',
                                                        'серпень',
                                                        'вересень',
                                                        'жовтень',
                                                        'листопад',
                                                        'грудень'])
  
  finish = finish.rename(columns={'variable': 'period', 'value': 'avg_salary'})
  finish = finish.astype({"tabNum": str}, errors='raise') 
  return finish

In [298]:
def f_54110(path_in):
    # Считывание файла справочника
    df_salary = pd.read_excel(path_in
                              , sheet_name='_54110'
                              , usecols='A,C:N'
                              , converters={'Таб №': str,
                                            'січень': float,
                                            'лютий': float,
                                            'березень': float,
                                            'квітень': float,
                                            'травень': float,
                                            'червень': float,
                                            'липень': float,
                                            'серпень': float,
                                            'вересень': float,
                                            'жовтень': float,
                                            'листопад': float,
                                            'грудень': float})
    # Преобразование таблицы из широкой в длинную для связывания с итоговой
    df_salary_unpivot = pd.melt(df_salary, id_vars='Таб. №', value_vars=['січень',
                                                                         'лютий',
                                                                         'березень',
                                                                         'квітень',
                                                                         'травень',
                                                                         'червень',
                                                                         'липень',
                                                                         'серпень',
                                                                         'вересень',
                                                                         'жовтень',
                                                                         'листопад',
                                                                         'грудень'])
    df_salary_unpivot = df_salary_unpivot.astype({"Таб. №": str}, errors='raise')
    return df_salary_unpivot

In [299]:
def f_54118(path_in):
    # Считывание файла справочника
    df_vacantion = pd.read_excel(path_in,
                                 sheet_name='_54118',
                                 usecols='A,C:N',
                                 converters={'Таб №': str,
                                             'січень': float,
                                             'лютий': float,
                                             'березень': float,
                                             'квітень': float,
                                             'травень': float,
                                             'червень': float,
                                             'липень': float,
                                             'серпень': float,
                                             'вересень': float,
                                             'жовтень': float,
                                             'листопад': float,
                                             'грудень': float})

    # Преобразование таблицы из широкой в длинную для связывания с итоговой
    df_vacantion_unpivot = pd.melt(df_vacantion, id_vars='Таб. №', value_vars=['січень',
                                                                               'лютий',
                                                                               'березень',
                                                                               'квітень',
                                                                               'травень',
                                                                               'червень',
                                                                               'липень',
                                                                               'серпень',
                                                                               'вересень',
                                                                               'жовтень',
                                                                               'листопад',
                                                                               'грудень'])

    df_salary_unpivot = df_vacantion_unpivot.astype({"Таб. №": str}, errors='raise')
    df_salary_unpivot = df_salary_unpivot['value'].fillna(0)
    
    return df_vacantion_unpivot

In [300]:
# Додаткова заробітна плата
def f_54112(path_in):
    # Считывание файла справочника
    df_add_salary = pd.read_excel(path_in
                                  , sheet_name='_54112'
                                  , usecols='B,G'
                                  , converters={'Таб. №': str,
                                                'итог': float})
    return df_add_salary

In [301]:
# Додаткова заробітна плата 5.4.1.1.3.3. премія за підсумками роботи Банку (за фінансовий рік)
def f_541133(path_in):
    # Считывание файла справочника
    df_add_salary_1 = pd.read_excel(path_in
                                    , sheet_name='_541133'
                                    , usecols='A,C:N').fillna(0)
    # Переворот таблицы из широкой в длинную
    df_add_salary_1 = pd.melt(df_add_salary_1, id_vars='Таб. №', value_vars=['січень',
                                                                             'лютий',
                                                                             'березень',
                                                                             'квітень',
                                                                             'травень',
                                                                             'червень',
                                                                             'липень',
                                                                             'серпень',
                                                                             'вересень',
                                                                             'жовтень',
                                                                             'листопад',
                                                                             'грудень'])
    df_add_salary_1 = df_add_salary_1.astype({"Таб. №": str}, errors='raise')
    
    df_add_salary_1 = df_add_salary_1.rename(columns={'Таб. №': 'tabNum',
                                                      'variable': 'period',
                                                      'value': 'add_salary_1'})

    
    return df_add_salary_1

In [302]:
# Додаткова заробітна плата 5.4.1.1.3.5. інші премії
def f_541135(path_in):
    # Считывание файла справочника
    df_add_salary_2 = pd.read_excel(path_in
                                    , sheet_name='_541135'
                                    , usecols='A,C:N').fillna(0)
    # Переворот таблицы из широкой в длинную
    df_add_salary_2 = pd.melt(df_add_salary_2, id_vars='Таб. №', value_vars=['січень',
                                                                             'лютий',
                                                                             'березень',
                                                                             'квітень',
                                                                             'травень',
                                                                             'червень',
                                                                             'липень',
                                                                             'серпень',
                                                                             'вересень',
                                                                             'жовтень',
                                                                             'листопад',
                                                                             'грудень'])
    df_add_salary_2 = df_add_salary_2.astype({"Таб. №": str}, errors='raise')
    df_add_salary_2 = df_add_salary_2.rename(columns={'Таб. №': 'tabNum',
                                                      'variable': 'period',
                                                      'value': 'add_salary_2'})

    return df_add_salary_2

In [303]:
# 5.4.1.1.4. трудові угоди
def f_54114(path_in):
    # Считывание файла справочника
    df_work_argmnt = pd.read_excel(path_in, sheet_name='_54114')

    df_work_argmnt = pd.melt(df_work_argmnt, id_vars='Таб. №', value_vars=['січень',
                                                                           'лютий',
                                                                           'березень',
                                                                           'квітень',
                                                                           'травень',
                                                                           'червень',
                                                                           'липень',
                                                                           'серпень',
                                                                           'вересень',
                                                                           'жовтень',
                                                                           'листопад',
                                                                           'грудень'])
    
    df_work_argmnt = df_work_argmnt.astype({"Таб. №": str}, errors='raise')

    df_work_argmnt = df_work_argmnt.rename(columns={'Таб. №': 'tabNum',
                                                    'variable': 'period',
                                                    'value': 'bug_work_argmnt'}).fillna(0)
    return df_work_argmnt

In [304]:
def f_5413(path_in):
    # Считывание файла справочника
    df_med_help = pd.read_excel(path_in, sheet_name='_5413', usecols='A:M', converters={'Місяць': str,
                                                                                        'січень': float,
                                                                                        'лютий': float,
                                                                                        'березень': float,
                                                                                        'квітень': float,
                                                                                        'травень': float,
                                                                                        'червень': float,
                                                                                        'липень': float,
                                                                                        'серпень': float,
                                                                                        'вересень': float,
                                                                                        'жовтень': float,
                                                                                        'листопад': float,
                                                                                        'грудень': float})

    df_med_help = pd.melt(df_med_help, id_vars='Місяць', value_vars=['січень',
                                                                     'лютий',
                                                                     'березень',
                                                                     'квітень',
                                                                     'травень',
                                                                     'червень',
                                                                     'липень',
                                                                     'серпень',
                                                                     'вересень',
                                                                     'жовтень',
                                                                     'листопад',
                                                                     'грудень'])
    df_med_help = df_med_help.rename(columns={'variable': 'period', 'value': 'bug_med_help'})
    del df_med_help['Місяць']

    return df_med_help

In [305]:
# 5.4.1.1.7. індексація
def f_54117(path_in):
    # Считывание файла справочника
    df_index = pd.read_excel(path_in, sheet_name='_54117', usecols='A:M')

    df_index = pd.melt(df_index, id_vars='Місяць', value_vars=['січень',
                                                               'лютий',
                                                               'березень',
                                                               'квітень',
                                                               'травень',
                                                               'червень',
                                                               'липень',
                                                               'серпень',
                                                               'вересень',
                                                               'жовтень',
                                                               'листопад',
                                                               'грудень'])
    df_index = df_index.rename(columns={'variable': 'period', 'value': 'bug_index'})
    del df_index['Місяць']

    return df_index

## **Розрахунок:**

### Щоб отримати щомісячний диапазон планування необхідно додати місяний розподіл:

In [306]:
months = ['січень', 'лютий', 'березень', 'квітень', 'травень', 'червень', 'липень', 'серпень', 'вересень', 'жовтень', 'листопад', 'грудень']

In [307]:
df_staff_month = []
for index, row in df_staff.iterrows():
  for l in months:
    df_staff_month.append({
        'tabNum': str(row['tabNum']),
        'period': str(l)})
df_staff = pd.DataFrame(df_staff_month).reindex()

In [308]:
# df_staff_for_merge.head()

In [309]:
# df_staff.head()

In [310]:
df_staff = pd.merge(left=df_staff, right=df_staff_for_merge, left_on='tabNum', right_on='tabNum', how='left')

In [311]:
#  заміна порожніх значень NaN на пробіл
df_staff['Name'] = df_staff['Name'].fillna('')

In [312]:
# df_staff.tail(12)

In [313]:
# df_staff.loc[df_staff.tabNum == 'вакансия_6']

### Доповнення розрахунком середньої зп (період розрахунку попрердні перед розрахунковим 6 місяців) відповідно до наданого файлу [(staff_in_avg.xlsx)](https://docs.google.com/spreadsheets/d/1hM5QhVZbz4R9yxtkOKcWNSCSPW9ebHOv/edit?usp=drive_web&ouid=114658159574667661980&rtpof=true)

In [314]:
path_in ='/content/gdrive/Shareddrives/Фінансовий департамент/Управління фінансового планування/Бюджет 2023/Budget FOP/In/staff_in_avg.xlsx'

до кожного табельного номеру розраховується середня зп за останні 6 місяців звітнього року за допомогою функції ***avg_salary***:

In [315]:
df_staff = pd.merge(df_staff, avg_salary(path_in), how='left', left_on=['tabNum', 'period'], right_on=['tabNum', 'period'])

In [316]:
#  заміна пустих значень NaN на 0 по колонці avg_salary
df_staff['avg_salary'] = df_staff['avg_salary'].fillna(0)

In [317]:
df_staff.loc[df_staff.tabNum == 'вакансия_5']

,tabNum,period,Name,CFOcode,CFOname,avg_salary


In [318]:
df_staff.loc[df_staff.tabNum == 'вакансия_6']

,tabNum,period,Name,CFOcode,CFOname,avg_salary


### Додавання планових зп на бюжетний рік, що надані в файлі [data_54110.xlsx](https://drive.google.com/drive/folders/175VRXdf4BPkIyLWTLRLhU9K4BLsc3D4I) за статею **5.4.1.1.0. "Основна заробітна плата (оклади) - бюджет"**

In [319]:
path_in = '/content/gdrive/Shareddrives/Фінансовий департамент/Управління фінансового планування/Бюджет 2023/Budget FOP/In/data_54110.xlsx'

In [320]:
# проміжна таблиця для приведення у відповідність назви столбців
df_staff_for_merge = f_54110(path_in).rename(columns={'Таб. №': 'tabNum', 'variable': 'period', 'value': 'bug_salary'})
# доповнення колонкою 'bug_salary' що відповідає статті 
df_staff = pd.merge(df_staff, df_staff_for_merge, how='left', left_on=['tabNum', 'period'], right_on=['tabNum', 'period'])
# заповнення пустых значень NaN на 0
df_staff['bug_salary'] = df_staff['bug_salary'].fillna(0)

In [321]:
# df_staff.head(12)

In [322]:
# df_staff.loc[df_staff.tabNum == 'вакансия_6']

### Додавання залишків відпусток (файл [data_54118.xlsx](https://drive.google.com/drive/folders/175VRXdf4BPkIyLWTLRLhU9K4BLsc3D4I)), як додаткову колонку ('vacantion') для використання в подальшому розрахунку

In [323]:
path_in = '/content/gdrive/Shareddrives/Фінансовий департамент/Управління фінансового планування/Бюджет 2023/Budget FOP/In/data_54118.xlsx'

In [324]:
df_staff_for_merge = f_54118(path_in).rename(columns={'Таб. №': 'tabNum', 'variable': 'period', 'value': 'vacantion'})
df_staff_for_merge['vacantion'] = df_staff_for_merge['vacantion'].fillna(0)
df_staff_for_merge = df_staff_for_merge.astype({"tabNum": str}, errors='raise')

In [325]:
df_staff.head()

,tabNum,period,Name,CFOcode,CFOname,avg_salary,bug_salary
0,8,січень,Багатько Сергій Миколайович,1.19.,Операційний департамент,1000.0,1000.0
1,8,лютий,Багатько Сергій Миколайович,1.19.,Операційний департамент,1000.0,1000.0
2,8,березень,Багатько Сергій Миколайович,1.19.,Операційний департамент,1000.0,1000.0
3,8,квітень,Багатько Сергій Миколайович,1.19.,Операційний департамент,1000.0,1000.0
4,8,травень,Багатько Сергій Миколайович,1.19.,Операційний департамент,1000.0,1000.0


In [326]:
df_staff = pd.merge(df_staff, df_staff_for_merge, how='left', left_on=['tabNum', 'period'], right_on=['tabNum','period'])

In [327]:
df_staff.head(12)

,tabNum,period,Name,CFOcode,CFOname,avg_salary,bug_salary,vacantion
0,8,січень,Багатько Сергій Миколайович,1.19.,Операційний департамент,1000.0,1000.0,NaN
1,8,лютий,Багатько Сергій Миколайович,1.19.,Операційний департамент,1000.0,1000.0,NaN
2,8,березень,Багатько Сергій Миколайович,1.19.,Операційний департамент,1000.0,1000.0,NaN
3,8,квітень,Багатько Сергій Миколайович,1.19.,Операційний департамент,1000.0,1000.0,NaN
4,8,травень,Багатько Сергій Миколайович,1.19.,Операційний департамент,1000.0,1000.0,NaN
5,8,червень,Багатько Сергій Миколайович,1.19.,Операційний департамент,1000.0,1000.0,NaN
6,8,липень,Багатько Сергій Миколайович,1.19.,Операційний департамент,1000.0,1000.0,NaN
7,8,серпень,Багатько Сергій Миколайович,1.19.,Операційний департамент,1000.0,1000.0,NaN
8,8,вересень,Багатько Сергій Миколайович,1.19.,Операційний департамент,1000.0,1000.0,NaN
9,8,жовтень,Багатько Сергій Миколайович,1.19.,Операційний департамент,1000.0,1000.0,NaN


### Показник **5.4.1.1.8. "Виплачені відпустки (3-й клас) - бюджет"** визначається, як добуток:
\begin{gather*}
{Виплачені\ відпустки (місяць)}_{бюджет} = ЗП\ середня(місяць)_{грн} \cdot Кількість(місяць)_{дн}
\end{gather*}

In [328]:
df_staff['bug_vacantion'] = df_staff['avg_salary'] * df_staff['vacantion']

In [329]:
df_staff.head(12)

,tabNum,period,Name,CFOcode,CFOname,avg_salary,bug_salary,vacantion,bug_vacantion
0,8,січень,Багатько Сергій Миколайович,1.19.,Операційний департамент,1000.0,1000.0,NaN,NaN
1,8,лютий,Багатько Сергій Миколайович,1.19.,Операційний департамент,1000.0,1000.0,NaN,NaN
2,8,березень,Багатько Сергій Миколайович,1.19.,Операційний департамент,1000.0,1000.0,NaN,NaN
3,8,квітень,Багатько Сергій Миколайович,1.19.,Операційний департамент,1000.0,1000.0,NaN,NaN
4,8,травень,Багатько Сергій Миколайович,1.19.,Операційний департамент,1000.0,1000.0,NaN,NaN
5,8,червень,Багатько Сергій Миколайович,1.19.,Операційний департамент,1000.0,1000.0,NaN,NaN
6,8,липень,Багатько Сергій Миколайович,1.19.,Операційний департамент,1000.0,1000.0,NaN,NaN
7,8,серпень,Багатько Сергій Миколайович,1.19.,Операційний департамент,1000.0,1000.0,NaN,NaN
8,8,вересень,Багатько Сергій Миколайович,1.19.,Операційний департамент,1000.0,1000.0,NaN,NaN
9,8,жовтень,Багатько Сергій Миколайович,1.19.,Операційний департамент,1000.0,1000.0,NaN,NaN


### Показник **5.4.1.1.1. "Основна заробітна плата"** розраховується, як різниця:
\begin{gather*}
{Основна\ заробітна\ плата(місяць)}_{бюджет} = {Основна\ заробітна\ плата/оклади\ (місяць)}_{бюджет} - {Виплачені\ відпустки (місяць)}_{бюджет} 
\end{gather*}

In [330]:
df_staff['bug_gen_salary'] = df_staff['bug_salary'] - df_staff['bug_vacantion']

In [331]:
df_staff.head(12)

,tabNum,period,Name,CFOcode,CFOname,avg_salary,bug_salary,vacantion,bug_vacantion,bug_gen_salary
0,8,січень,Багатько Сергій Миколайович,1.19.,Операційний департамент,1000.0,1000.0,NaN,NaN,NaN
1,8,лютий,Багатько Сергій Миколайович,1.19.,Операційний департамент,1000.0,1000.0,NaN,NaN,NaN
2,8,березень,Багатько Сергій Миколайович,1.19.,Операційний департамент,1000.0,1000.0,NaN,NaN,NaN
3,8,квітень,Багатько Сергій Миколайович,1.19.,Операційний департамент,1000.0,1000.0,NaN,NaN,NaN
4,8,травень,Багатько Сергій Миколайович,1.19.,Операційний департамент,1000.0,1000.0,NaN,NaN,NaN
5,8,червень,Багатько Сергій Миколайович,1.19.,Операційний департамент,1000.0,1000.0,NaN,NaN,NaN
6,8,липень,Багатько Сергій Миколайович,1.19.,Операційний департамент,1000.0,1000.0,NaN,NaN,NaN
7,8,серпень,Багатько Сергій Миколайович,1.19.,Операційний департамент,1000.0,1000.0,NaN,NaN,NaN
8,8,вересень,Багатько Сергій Миколайович,1.19.,Операційний департамент,1000.0,1000.0,NaN,NaN,NaN
9,8,жовтень,Багатько Сергій Миколайович,1.19.,Операційний департамент,1000.0,1000.0,NaN,NaN,NaN


### Показник **5.4.1.1.2. Додаткова заробітна плата**, визначається з файлу [data_54112.xlsx](https://drive.google.com/drive/folders/175VRXdf4BPkIyLWTLRLhU9K4BLsc3D4I)

In [332]:
path_in = '/content/gdrive/Shareddrives/Фінансовий департамент/Управління фінансового планування/Бюджет 2023/Budget FOP/In/data_54112.xlsx'

In [333]:
#
df_staff_for_merge = f_54112(path_in).rename(columns={'Таб. №': 'tabNum', 'итог': 'bug_add_salary'})
df_staff_for_merge['bug_add_salary'] = df_staff_for_merge['bug_add_salary'].fillna(0)
df_staff = df_staff.merge(df_staff_for_merge, on='tabNum', how='left').fillna(0)

In [334]:
df_staff.head(12)

,tabNum,period,Name,CFOcode,CFOname,avg_salary,bug_salary,vacantion,bug_vacantion,bug_gen_salary,bug_add_salary
0,8,січень,Багатько Сергій Миколайович,1.19.,Операційний департамент,1000.0,1000.0,0,0,0,41600.0
1,8,лютий,Багатько Сергій Миколайович,1.19.,Операційний департамент,1000.0,1000.0,0,0,0,41600.0
2,8,березень,Багатько Сергій Миколайович,1.19.,Операційний департамент,1000.0,1000.0,0,0,0,41600.0
3,8,квітень,Багатько Сергій Миколайович,1.19.,Операційний департамент,1000.0,1000.0,0,0,0,41600.0
4,8,травень,Багатько Сергій Миколайович,1.19.,Операційний департамент,1000.0,1000.0,0,0,0,41600.0
5,8,червень,Багатько Сергій Миколайович,1.19.,Операційний департамент,1000.0,1000.0,0,0,0,41600.0
6,8,липень,Багатько Сергій Миколайович,1.19.,Операційний департамент,1000.0,1000.0,0,0,0,41600.0
7,8,серпень,Багатько Сергій Миколайович,1.19.,Операційний департамент,1000.0,1000.0,0,0,0,41600.0
8,8,вересень,Багатько Сергій Миколайович,1.19.,Операційний департамент,1000.0,1000.0,0,0,0,41600.0
9,8,жовтень,Багатько Сергій Миколайович,1.19.,Операційний департамент,1000.0,1000.0,0,0,0,41600.0


### Показник **5.4.1.1.3.3. премія за підсумками роботи Банку (за фінансовий рік)** визначається з файлу [data_541133.xlsx](https://drive.google.com/drive/folders/175VRXdf4BPkIyLWTLRLhU9K4BLsc3D4I)

In [335]:
path_in = '/content/gdrive/Shareddrives/Фінансовий департамент/Управління фінансового планування/Бюджет 2023/Budget FOP/In/data_541133.xlsx'

In [336]:
df_staff = pd.merge(df_staff, f_541133(path_in), how='left', left_on=['tabNum', 'period'], right_on=['tabNum', 'period'])

In [337]:
df_staff.head(12)

,tabNum,period,Name,CFOcode,CFOname,avg_salary,bug_salary,vacantion,bug_vacantion,bug_gen_salary,bug_add_salary,add_salary_1
0,8,січень,Багатько Сергій Миколайович,1.19.,Операційний департамент,1000.0,1000.0,0,0,0,41600.0,NaN
1,8,лютий,Багатько Сергій Миколайович,1.19.,Операційний департамент,1000.0,1000.0,0,0,0,41600.0,NaN
2,8,березень,Багатько Сергій Миколайович,1.19.,Операційний департамент,1000.0,1000.0,0,0,0,41600.0,NaN
3,8,квітень,Багатько Сергій Миколайович,1.19.,Операційний департамент,1000.0,1000.0,0,0,0,41600.0,NaN
4,8,травень,Багатько Сергій Миколайович,1.19.,Операційний департамент,1000.0,1000.0,0,0,0,41600.0,NaN
5,8,червень,Багатько Сергій Миколайович,1.19.,Операційний департамент,1000.0,1000.0,0,0,0,41600.0,NaN
6,8,липень,Багатько Сергій Миколайович,1.19.,Операційний департамент,1000.0,1000.0,0,0,0,41600.0,NaN
7,8,серпень,Багатько Сергій Миколайович,1.19.,Операційний департамент,1000.0,1000.0,0,0,0,41600.0,NaN
8,8,вересень,Багатько Сергій Миколайович,1.19.,Операційний департамент,1000.0,1000.0,0,0,0,41600.0,NaN
9,8,жовтень,Багатько Сергій Миколайович,1.19.,Операційний департамент,1000.0,1000.0,0,0,0,41600.0,NaN


###Показник **5.4.1.1.3.5. Інші премії** визначається з файлу [data_541135.xlsx](https://drive.google.com/drive/folders/175VRXdf4BPkIyLWTLRLhU9K4BLsc3D4I)

In [338]:
path_in = '/content/gdrive/Shareddrives/Фінансовий департамент/Управління фінансового планування/Бюджет 2023/Budget FOP/In/data_541135.xlsx'

In [339]:
df_staff = pd.merge(df_staff, f_541135(path_in), how='left', left_on=['tabNum', 'period'],
                    right_on=['tabNum',
                              'period'])

In [340]:
df_staff.head(12)

,tabNum,period,Name,CFOcode,CFOname,avg_salary,bug_salary,vacantion,bug_vacantion,bug_gen_salary,bug_add_salary,add_salary_1,add_salary_2
0,8,січень,Багатько Сергій Миколайович,1.19.,Операційний департамент,1000.0,1000.0,0,0,0,41600.0,NaN,NaN
1,8,лютий,Багатько Сергій Миколайович,1.19.,Операційний департамент,1000.0,1000.0,0,0,0,41600.0,NaN,NaN
2,8,березень,Багатько Сергій Миколайович,1.19.,Операційний департамент,1000.0,1000.0,0,0,0,41600.0,NaN,NaN
3,8,квітень,Багатько Сергій Миколайович,1.19.,Операційний департамент,1000.0,1000.0,0,0,0,41600.0,NaN,NaN
4,8,травень,Багатько Сергій Миколайович,1.19.,Операційний департамент,1000.0,1000.0,0,0,0,41600.0,NaN,NaN
5,8,червень,Багатько Сергій Миколайович,1.19.,Операційний департамент,1000.0,1000.0,0,0,0,41600.0,NaN,NaN
6,8,липень,Багатько Сергій Миколайович,1.19.,Операційний департамент,1000.0,1000.0,0,0,0,41600.0,NaN,NaN
7,8,серпень,Багатько Сергій Миколайович,1.19.,Операційний департамент,1000.0,1000.0,0,0,0,41600.0,NaN,NaN
8,8,вересень,Багатько Сергій Миколайович,1.19.,Операційний департамент,1000.0,1000.0,0,0,0,41600.0,NaN,NaN
9,8,жовтень,Багатько Сергій Миколайович,1.19.,Операційний департамент,1000.0,1000.0,0,0,0,41600.0,NaN,NaN


###Показник **5.4.1.1.5. Резерв відпусток (7 кл.)** визначається за допомогою довідника [data_54115.xlsx](https://drive.google.com/drive/folders/175VRXdf4BPkIyLWTLRLhU9K4BLsc3D4I)

In [341]:
path_in = '/content/gdrive/Shareddrives/Фінансовий департамент/Управління фінансового планування/Бюджет 2023/Budget FOP/In/data_54115.xlsx'

In [342]:
# Завантаження довідника
df_add_vacant = pd.read_excel(path_in, sheet_name='_54115', usecols='A,E').fillna(0)
# Приведення у відповідність назв вимірів
df_add_vacant = df_add_vacant.rename(columns={'Таб. №': 'tabNum', 'Днів': 'bug_vac_reserv'})
df_add_vacant['tabNum'] = df_add_vacant['tabNum'].astype('str').replace('.0', '')
# df_add_vacant['bug_vac_reserv'] = df_add_vacant['bug_vac_reserv'].astype('int')
# Додавання виміру до кінцевого датасета
df_staff = df_staff.merge(df_add_vacant, on='tabNum', how='left').fillna(0)

In [343]:
df_add_vacant.dtypes

tabNum             object
bug_vac_reserv    float64
dtype: object

In [344]:
df_staff.head(12)

,tabNum,period,Name,CFOcode,CFOname,avg_salary,bug_salary,vacantion,bug_vacantion,bug_gen_salary,bug_add_salary,add_salary_1,add_salary_2,bug_vac_reserv
0,8,січень,Багатько Сергій Миколайович,1.19.,Операційний департамент,1000.0,1000.0,0,0,0,41600.0,0.0,0.0,0.0
1,8,лютий,Багатько Сергій Миколайович,1.19.,Операційний департамент,1000.0,1000.0,0,0,0,41600.0,0.0,0.0,0.0
2,8,березень,Багатько Сергій Миколайович,1.19.,Операційний департамент,1000.0,1000.0,0,0,0,41600.0,0.0,0.0,0.0
3,8,квітень,Багатько Сергій Миколайович,1.19.,Операційний департамент,1000.0,1000.0,0,0,0,41600.0,0.0,0.0,0.0
4,8,травень,Багатько Сергій Миколайович,1.19.,Операційний департамент,1000.0,1000.0,0,0,0,41600.0,0.0,0.0,0.0
5,8,червень,Багатько Сергій Миколайович,1.19.,Операційний департамент,1000.0,1000.0,0,0,0,41600.0,0.0,0.0,0.0
6,8,липень,Багатько Сергій Миколайович,1.19.,Операційний департамент,1000.0,1000.0,0,0,0,41600.0,0.0,0.0,0.0
7,8,серпень,Багатько Сергій Миколайович,1.19.,Операційний департамент,1000.0,1000.0,0,0,0,41600.0,0.0,0.0,0.0
8,8,вересень,Багатько Сергій Миколайович,1.19.,Операційний департамент,1000.0,1000.0,0,0,0,41600.0,0.0,0.0,0.0
9,8,жовтень,Багатько Сергій Миколайович,1.19.,Операційний департамент,1000.0,1000.0,0,0,0,41600.0,0.0,0.0,0.0


### Показник **5.4.1.1.4. трудові угоди** визначається за допомогою довідника [data_54114.xlsx](https://drive.google.com/drive/folders/175VRXdf4BPkIyLWTLRLhU9K4BLsc3D4I)

In [345]:
path_in = '/content/gdrive/Shareddrives/Фінансовий департамент/Управління фінансового планування/Бюджет 2023/Budget FOP/In/data_54114.xlsx'

In [346]:
df_staff = pd.merge(df_staff, f_54114(path_in), how='left', left_on=['tabNum', 'period'], right_on=['tabNum', 'period'])

In [347]:
df_staff.head(12)

,tabNum,period,Name,CFOcode,CFOname,avg_salary,bug_salary,vacantion,bug_vacantion,bug_gen_salary,bug_add_salary,add_salary_1,add_salary_2,bug_vac_reserv,bug_work_argmnt
0,8,січень,Багатько Сергій Миколайович,1.19.,Операційний департамент,1000.0,1000.0,0,0,0,41600.0,0.0,0.0,0.0,NaN
1,8,лютий,Багатько Сергій Миколайович,1.19.,Операційний департамент,1000.0,1000.0,0,0,0,41600.0,0.0,0.0,0.0,NaN
2,8,березень,Багатько Сергій Миколайович,1.19.,Операційний департамент,1000.0,1000.0,0,0,0,41600.0,0.0,0.0,0.0,NaN
3,8,квітень,Багатько Сергій Миколайович,1.19.,Операційний департамент,1000.0,1000.0,0,0,0,41600.0,0.0,0.0,0.0,NaN
4,8,травень,Багатько Сергій Миколайович,1.19.,Операційний департамент,1000.0,1000.0,0,0,0,41600.0,0.0,0.0,0.0,NaN
5,8,червень,Багатько Сергій Миколайович,1.19.,Операційний департамент,1000.0,1000.0,0,0,0,41600.0,0.0,0.0,0.0,NaN
6,8,липень,Багатько Сергій Миколайович,1.19.,Операційний департамент,1000.0,1000.0,0,0,0,41600.0,0.0,0.0,0.0,NaN
7,8,серпень,Багатько Сергій Миколайович,1.19.,Операційний департамент,1000.0,1000.0,0,0,0,41600.0,0.0,0.0,0.0,NaN
8,8,вересень,Багатько Сергій Миколайович,1.19.,Операційний департамент,1000.0,1000.0,0,0,0,41600.0,0.0,0.0,0.0,NaN
9,8,жовтень,Багатько Сергій Миколайович,1.19.,Операційний департамент,1000.0,1000.0,0,0,0,41600.0,0.0,0.0,0.0,NaN


### Показник **5.4.1.3. мат. допомога і інші утримання** визначається за допомогою довідника [data_5413.xlsx](https://drive.google.com/drive/folders/175VRXdf4BPkIyLWTLRLhU9K4BLsc3D4I)

In [348]:
path_in = '/content/gdrive/Shareddrives/Фінансовий департамент/Управління фінансового планування/Бюджет 2023/Budget FOP/In/data_5413.xlsx'

In [349]:
# Додавання виміру мат.допомоги до кінцевого датасету
df_staff = df_staff.merge(f_5413(path_in), on='period', how='left').fillna(0)

In [350]:
df_staff.head(12)

,tabNum,period,Name,CFOcode,CFOname,avg_salary,bug_salary,vacantion,bug_vacantion,bug_gen_salary,bug_add_salary,add_salary_1,add_salary_2,bug_vac_reserv,bug_work_argmnt,bug_med_help
0,8,січень,Багатько Сергій Миколайович,1.19.,Операційний департамент,1000.0,1000.0,0,0,0,41600.0,0.0,0.0,0.0,0.0,32000.0
1,8,січень,Багатько Сергій Миколайович,1.19.,Операційний департамент,1000.0,1000.0,0,0,0,41600.0,0.0,0.0,0.0,0.0,0.0
2,8,січень,Багатько Сергій Миколайович,1.19.,Операційний департамент,1000.0,1000.0,0,0,0,41600.0,0.0,0.0,0.0,0.0,0.0
3,8,січень,Багатько Сергій Миколайович,1.19.,Операційний департамент,1000.0,1000.0,0,0,0,41600.0,0.0,0.0,0.0,0.0,500000.0
4,8,лютий,Багатько Сергій Миколайович,1.19.,Операційний департамент,1000.0,1000.0,0,0,0,41600.0,0.0,0.0,0.0,0.0,32000.0
5,8,лютий,Багатько Сергій Миколайович,1.19.,Операційний департамент,1000.0,1000.0,0,0,0,41600.0,0.0,0.0,0.0,0.0,0.0
6,8,лютий,Багатько Сергій Миколайович,1.19.,Операційний департамент,1000.0,1000.0,0,0,0,41600.0,0.0,0.0,0.0,0.0,0.0
7,8,лютий,Багатько Сергій Миколайович,1.19.,Операційний департамент,1000.0,1000.0,0,0,0,41600.0,0.0,0.0,0.0,0.0,500000.0
8,8,березень,Багатько Сергій Миколайович,1.19.,Операційний департамент,1000.0,1000.0,0,0,0,41600.0,0.0,0.0,0.0,0.0,32000.0
9,8,березень,Багатько Сергій Миколайович,1.19.,Операційний департамент,1000.0,1000.0,0,0,0,41600.0,0.0,0.0,0.0,0.0,62500.0


### Проміжний розрахунок індексації за допомогою файлу [data_54117.xlsx](https://drive.google.com/drive/folders/175VRXdf4BPkIyLWTLRLhU9K4BLsc3D4I)

In [351]:
path_in = '/content/gdrive/Shareddrives/Фінансовий департамент/Управління фінансового планування/Бюджет 2023/Budget FOP/In/data_54117.xlsx'

In [352]:
# Додавання значення індексу в розрахункову таблицю
df_staff = df_staff.merge(f_54117(path_in), on='period', how='left').fillna(0)

### Показник **5.4.1.1.7. Індексація** (остаточний розрахунок):

In [353]:
# ???? уточнить формулу с учетом 5.4.1.3.	мат. допомога і інші утримання
df_staff['bug_indexing'] = (df_staff['bug_salary']
                            + df_staff['bug_add_salary']
                            + df_staff['add_salary_1']
                            + df_staff['add_salary_2']
                            + df_staff['bug_work_argmnt']
                            + df_staff['bug_med_help']) * df_staff['bug_index']

In [354]:
df_staff.head(12)

,tabNum,period,Name,CFOcode,CFOname,avg_salary,bug_salary,vacantion,bug_vacantion,bug_gen_salary,bug_add_salary,add_salary_1,add_salary_2,bug_vac_reserv,bug_work_argmnt,bug_med_help,bug_index,bug_indexing
0,8,січень,Багатько Сергій Миколайович,1.19.,Операційний департамент,1000.0,1000.0,0,0,0,41600.0,0.0,0.0,0.0,0.0,32000.0,0.00832,620.672
1,8,січень,Багатько Сергій Миколайович,1.19.,Операційний департамент,1000.0,1000.0,0,0,0,41600.0,0.0,0.0,0.0,0.0,0.0,0.00832,354.432
2,8,січень,Багатько Сергій Миколайович,1.19.,Операційний департамент,1000.0,1000.0,0,0,0,41600.0,0.0,0.0,0.0,0.0,0.0,0.00832,354.432
3,8,січень,Багатько Сергій Миколайович,1.19.,Операційний департамент,1000.0,1000.0,0,0,0,41600.0,0.0,0.0,0.0,0.0,500000.0,0.00832,4514.432
4,8,лютий,Багатько Сергій Миколайович,1.19.,Операційний департамент,1000.0,1000.0,0,0,0,41600.0,0.0,0.0,0.0,0.0,32000.0,0.00820,611.720
5,8,лютий,Багатько Сергій Миколайович,1.19.,Операційний департамент,1000.0,1000.0,0,0,0,41600.0,0.0,0.0,0.0,0.0,0.0,0.00820,349.320
6,8,лютий,Багатько Сергій Миколайович,1.19.,Операційний департамент,1000.0,1000.0,0,0,0,41600.0,0.0,0.0,0.0,0.0,0.0,0.00820,349.320
7,8,лютий,Багатько Сергій Миколайович,1.19.,Операційний департамент,1000.0,1000.0,0,0,0,41600.0,0.0,0.0,0.0,0.0,500000.0,0.00820,4449.320
8,8,березень,Багатько Сергій Миколайович,1.19.,Операційний департамент,1000.0,1000.0,0,0,0,41600.0,0.0,0.0,0.0,0.0,32000.0,0.00805,600.530
9,8,березень,Багатько Сергій Миколайович,1.19.,Операційний департамент,1000.0,1000.0,0,0,0,41600.0,0.0,0.0,0.0,0.0,62500.0,0.00805,846.055


### Показник **5.4.1.2.1. Обов'язкове державне пенсійне та соціальне страхування** 

In [355]:
var_54117 = 7700*15     # ХЗ что за переменная
df_staff['bug_gov_insurance'] = np.where((df_staff['bug_salary']
                                          + df_staff['bug_add_salary']
                                          + df_staff['add_salary_1']
                                          + df_staff['add_salary_2']
                                          + df_staff['bug_work_argmnt']
                                          + df_staff['bug_med_help']) <= var_54117,
                                         (df_staff['bug_salary']
                                          + df_staff['bug_add_salary']
                                          + df_staff['add_salary_1']
                                          + df_staff['add_salary_2']
                                          + df_staff['bug_work_argmnt']
                                          + df_staff['bug_med_help']) * 0.22,
                                         var_54117 * 0.22) + df_staff['bug_vac_reserv'] * 0.22

### Показник **5.4.1.5.8. профспілкові відрахування**

In [356]:
# Визначення видношення до профспілки
df_staff['bug_labour_union'] = np.where(df_staff['tabNum'].str.startswith(('труд'), na=False), 0, (df_staff['bug_salary']
                                                                                                   + df_staff['bug_add_salary']
                                                                                                   + df_staff['add_salary_1']
                                                                                                   + df_staff['add_salary_2']
                                                                                                   + df_staff['bug_vac_reserv']
                                                                                                   + df_staff['bug_indexing']
                                                                                                   + df_staff['bug_med_help']
                                                                                                   + df_staff['bug_gov_insurance']) * 0.05)

In [357]:
df_staff['bug_labour_union'].loc[df_staff.tabNum == 'труд_4']

178992    0.0
178993    0.0
178994    0.0
178995    0.0
178996    0.0
178997    0.0
178998    0.0
178999    0.0
179000    0.0
179001    0.0
179002    0.0
179003    0.0
179004    0.0
179005    0.0
179006    0.0
179007    0.0
179008    0.0
179009    0.0
179010    0.0
179011    0.0
179012    0.0
179013    0.0
179014    0.0
179015    0.0
179016    0.0
179017    0.0
179018    0.0
179019    0.0
179020    0.0
179021    0.0
179022    0.0
179023    0.0
179024    0.0
179025    0.0
179026    0.0
179027    0.0
179028    0.0
179029    0.0
179030    0.0
179031    0.0
179032    0.0
179033    0.0
179034    0.0
179035    0.0
179036    0.0
179037    0.0
179038    0.0
179039    0.0
Name: bug_labour_union, dtype: float64

### Для зберігання остаточного розрахунку необхідно здійснити заміну імен вимірів

In [358]:
df_staff = df_staff.rename(columns={'tabNum': 'Таб №',
                                        'avg_salary': 'Середня зп звітнього року',
                                        'period': 'Місяць бюджетного року',
                                        'Name': 'ПІБ',
                                        'CFOcode': 'ЦФУ код',
                                        'CFOname': 'ЦФУ назва',
                                        'bug_salary': '5.4.1.1.0.',
                                        'vacantion': 'Планові відпустки, дн. (в)',
                                        'bug_vacantion': '5.4.1.1.8.',
                                        'bug_gen_salary': '5.4.1.1.1.',
                                        'bug_add_salary': '5.4.1.1.2.',
                                        'add_salary_1': '5.4.1.1.3.3.',
                                        'add_salary_2': '5.4.1.1.3.5.',
                                        'bug_vac_reserv': '5.4.1.1.5.',
                                        'bug_work_argmnt': '5.4.1.1.4.',
                                        'bug_med_help': '5.4.1.3.',
                                        'bug_index': 'індекс (в)',
                                        'bug_indexing': '5.4.1.1.7.',
                                        'bug_gov_insurance': '5.4.1.2.1.',
                                        'bug_labour_union': '5.4.1.5.8.',
                                        })

### Попередній вигляд остаточного вигяду розрахунку перед зберіганням

In [359]:
print(df_staff.head(2))

  Таб № Місяць бюджетного року                          ПІБ ЦФУ код  \
0     8                 січень  Багатько Сергій Миколайович   1.19.   
1     8                 січень  Багатько Сергій Миколайович   1.19.   

                 ЦФУ назва  Середня зп звітнього року  5.4.1.1.0.  \
0  Операційний департамент                     1000.0      1000.0   
1  Операційний департамент                     1000.0      1000.0   

   Планові відпустки, дн. (в)  5.4.1.1.8.  5.4.1.1.1.  5.4.1.1.2.  \
0                           0           0           0     41600.0   
1                           0           0           0     41600.0   

   5.4.1.1.3.3.  5.4.1.1.3.5.  5.4.1.1.5.  5.4.1.1.4.  5.4.1.3.  індекс (в)  \
0           0.0           0.0         0.0         0.0   32000.0     0.00832   
1           0.0           0.0         0.0         0.0       0.0     0.00832   

   5.4.1.1.7.  5.4.1.2.1.  5.4.1.5.8.  
0     620.672     16412.0   4581.6336  
1     354.432      9372.0   2616.3216  


In [360]:
df_staff = df_staff.drop(['Середня зп звітнього року', 'Планові відпустки, дн. (в)', 'індекс (в)'], axis=1)

In [361]:
df_staff.columns

Index(['Таб №', 'Місяць бюджетного року', 'ПІБ', 'ЦФУ код', 'ЦФУ назва',
       '5.4.1.1.0.', '5.4.1.1.8.', '5.4.1.1.1.', '5.4.1.1.2.', '5.4.1.1.3.3.',
       '5.4.1.1.3.5.', '5.4.1.1.5.', '5.4.1.1.4.', '5.4.1.3.', '5.4.1.1.7.',
       '5.4.1.2.1.', '5.4.1.5.8.'],
      dtype='object')

##**Зберігання результуючого датасету до папки "Out"**

In [362]:
path_out = '/content/gdrive/Shareddrives/Фінансовий департамент/Управління фінансового планування/Бюджет 2023/Budget FOP/Out/fop.xlsx'

In [363]:
df_staff.to_excel(path_out)

In [364]:
print("Calculation Done! 🎉 ")

Calculation Done! 🎉 


##**Форматування файлу для SA**

In [365]:
path_in ='/content/gdrive/Shareddrives/Фінансовий департамент/Управління фінансового планування/Бюджет 2023/Budget FOP/Out/fop.xlsx'

In [366]:
sheet_df_dictonary = pd.read_excel(path_in, engine='openpyxl', sheet_name=['Sheet1'], skiprows=0)

In [367]:
Sheet1 = sheet_df_dictonary['Sheet1'].drop(['Unnamed: 0'], axis=1)

In [368]:
print(Sheet1.columns)

Index(['Таб №', 'Місяць бюджетного року', 'ПІБ', 'ЦФУ код', 'ЦФУ назва',
       '5.4.1.1.0.', '5.4.1.1.8.', '5.4.1.1.1.', '5.4.1.1.2.', '5.4.1.1.3.3.',
       '5.4.1.1.3.5.', '5.4.1.1.5.', '5.4.1.1.4.', '5.4.1.3.', '5.4.1.1.7.',
       '5.4.1.2.1.', '5.4.1.5.8.'],
      dtype='object')


In [369]:
def fResultForUploadSA(vArticleCode, vArticleName):
  # Виборка статті для агрегації
  df_new = Sheet1[['Місяць бюджетного року', 'ЦФУ код', 'ЦФУ назва', vArticleCode]].copy()
  # Початковий датафрейм
  df_tmp = df_new[df_new['Місяць бюджетного року'] == 'січень']
  result = df_tmp.groupby(['ЦФУ код', 'ЦФУ назва'])[vArticleCode].aggregate('sum').to_frame().reset_index()
  #
  result.insert(0, 'Назва статті', vArticleName)
  result.insert(0, 'Код статті', vArticleCode)
  # 
  result_total = result.rename(columns = {vArticleCode:'січень'})
  #
  months = ['лютий', 'березень', 'квітень', 'травень', 'червень', 'липень', 'серпень', 'вересень', 'жовтень', 'листопад', 'грудень']
  for i in months:
    df_new_tmp = df_new[df_new['Місяць бюджетного року'] == i]
    result = df_new_tmp.groupby(['ЦФУ код', 'ЦФУ назва'])[vArticleCode].aggregate('sum').to_frame().reset_index()
    result.insert(0, 'Назва статті', vArticleName)
    result.insert(0, 'Код статті', vArticleCode)
    df_tmp = result.rename(columns = {vArticleCode: i})
    result_total = pd.merge(result_total, df_tmp)

  return result_total

In [370]:
cluster_1 = fResultForUploadSA('5.4.1.1.0.','основна заробітна плата (оклади) - бюджет')
cluster_2 = fResultForUploadSA('5.4.1.1.8.','Виплачені відпустки (3-й клас) - бюджет')
cluster_3 = fResultForUploadSA('5.4.1.1.1.','основна заробітна плата')
cluster_4 = fResultForUploadSA('5.4.1.1.2.','додаткова заробітна плата')
cluster_5 = fResultForUploadSA('5.4.1.1.3.3.','премія за підсумками роботи Банку (за фінансовий рік)')
cluster_6 = fResultForUploadSA('5.4.1.1.3.5.','інші премії')
cluster_7 = fResultForUploadSA('5.4.1.1.5.','резерв відпусток')
cluster_8 = fResultForUploadSA('5.4.1.1.4.','трудові угоди')
cluster_9 = fResultForUploadSA('5.4.1.3.','мат. допомога і інші утримання')
cluster_10 = fResultForUploadSA('5.4.1.1.7.','індексація')
cluster_11 = fResultForUploadSA('5.4.1.2.1.','Обов\'язкове державне пенсійне та соціальне страхування')
cluster_12 = fResultForUploadSA('5.4.1.5.8.','профспілкові відрахування')

In [371]:
pdList = [cluster_1, cluster_2, cluster_3, cluster_4, cluster_5, cluster_6, cluster_7, cluster_8, cluster_9, cluster_10, cluster_11, cluster_12]
table = pd.concat(pdList)

## Доповнення стовбців для шаблону

In [372]:
table.insert(4, 'грудень 2022', '')
table.insert(4, 'Код проекта','')
table.insert(2, 'Код бізнеса','')
table.insert(2, 'Валюта', '[UAH]')
table.insert(2, 'Назва підрозділу', '')
table.insert(2, 'Код підрозділу', '')
table.insert(2, 'Назва дирекції','')
table.insert(2, 'Код дирекції','')
table.insert(0, 'Назва ініціатора','')
table.insert(0, 'Код ініціатора','1.42.')
table.insert(0, 'Назва адміністратора','')
table.insert(0, 'Код адміністратора', '1.43.')

In [373]:
# table.head()

In [374]:
path_out = '/content/gdrive/Shareddrives/Фінансовий департамент/Управління фінансового планування/Бюджет 2023/Budget FOP/Out/fop_SA.xlsx'

In [375]:
table.to_excel(path_out, index=False)

In [376]:
print("Formatting Done! 🎉 ")

Formatting Done! 🎉 
